## Simple Custom Agent 

### Imports

In [1]:

import json
import sys
sys.path.append('TinyTroupe')

import tinytroupe
from tinytroupe.agent import TinyPerson
from tinytroupe.examples import create_lisa_the_data_scientist, create_oscar_the_architect
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.agent import RecallFaculty, FilesAndWebGroundingFaculty

from tinytroupe.extraction import ResultsExtractor
import logging
logging.getLogger("tinytroupe").setLevel(logging.WARNING)  # or logging.ERROR for even less output

from dotenv import load_dotenv
import os

load_dotenv()

ModuleNotFoundError: No module named 'rich'

---

### Simplified Custom agents- Sai

Custom agent - will ask him or her to perform the evaluations. To make it easier to change the chosen agent, assigning it to a variable first.

In [2]:
print(list(TinyPerson.all_agents.keys()))

NameError: name 'TinyPerson' is not defined

In [44]:
TinyPerson.all_agents.clear()
print(list(TinyPerson.all_agents.keys()))

[]


In [45]:
# Then create the new Lisa
agent_sai = TinyPerson("Sai Abhishek")

```
sai = TinyPerson(
    name="Sai Abhishek",
    episodic_memory=EpisodicMemory(fixed_prefix_length=100, lookback_length=100),
    semantic_memory=SemanticMemory(documents_paths=["/path/to/docs"], web_urls=["https://example.com"]),
    mental_faculties=[RecallFaculty(), FilesAndWebGroundingFaculty()]
)
```

1. The TinyPerson class is the main agent class that simulates human-like behavior with:
- Cognitive states (attention, emotions, goals)
- Memory systems (episodic and semantic)
- Interaction capabilities (listening, speaking, seeing, thinking)
- Mental faculties (abilities like recall, tool use, file access)
2. The memory system has two main components:
- EpisodicMemory: Stores temporal sequences of events/experiences
- SemanticMemory: Stores general knowledge and facts, with support for document storage and retrieval
3. Mental faculties like RecallFaculty, FilesAndWebGroundingFaculty, and TinyToolUse define different cognitive abilities that agents can have.
4. The agent architecture follows psychological principles:
- Stimulus-response model for interaction
- Separate episodic and semantic memory systems
- Internal cognitive states that influence behavior
- Goal-directed behavior capabilities

In [46]:
# Defining basic traits
agent_sai.define("age", 27)
agent_sai.define("nationality", "Indian")
agent_sai.define("occupation", "AI Architect")

In [47]:
# Add mental faculties
agent_sai.add_mental_faculties([
    RecallFaculty(),
    FilesAndWebGroundingFaculty()
])


TinyPerson(name='Sai Abhishek')

In [48]:
# Define more traits
agent_sai.define("professional_interests", ["AI", "Machine Learning", "Architecture"])
agent_sai.define("skills", ["Python", "Deep Learning", "System Design"])

In [49]:
# Start interacting
agent_sai.listen("Hello, can you tell me about yourself?")
agent_sai.act()

USER --> Sai Abhishek: [CONVERSATION] 
          > Hello, can you tell me about yourself?

Sai Abhishek acts: [THINK] 
                  > I should introduce myself in a friendly and concise manner.

Sai Abhishek acts: [TALK] 
                  > Hello! My name is Sai Abhishek. I'm 27 years old and I work as an AI Architect.
                  > I'm originally from India.

Sai Abhishek acts: [DONE] 

---

# Online Advertisement Evaluation for TVs

Let's evaluate some online ads options to pick the best one.

## Judging the best ad

Some ads data, obtained from real Bing queries.

In [61]:
# User search query: "55 inches tv"

tv_ad_1 =\
"""
The Best TV Of Tomorrow - LG 4K Ultra HD TV
https://www.lg.com/tv/oled
AdThe Leading Name in Cinematic Picture. Upgrade Your TV to 4K OLED And See The Difference. It's Not Just OLED, It's LG OLED. Exclusive a9 Processor, Bringing Cinematic Picture Home.

Infinite Contrast · Self-Lighting OLED · Dolby Vision™ IQ · ThinQ AI w/ Magic Remote

Free Wall Mounting Deal
LG G2 97" OLED evo TV
Free TV Stand w/ Purchase
World's No.1 OLED TV
"""

tv_ad_2 =\
"""
The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More
https://www.samsung.com
AdFrom 4K To 8K, QLED To OLED, Lifestyle TVs & More, Your Perfect TV Is In Our Lineup. Experience Unrivaled Technology & Design In Our Ultra-Premium 8K & 4K TVs.

Discover Samsung Event · Real Depth Enhancer · Anti-Reflection · 48 mo 0% APR Financing

The 2023 OLED TV Is Here
Samsung Neo QLED 4K TVs
Samsung Financing
Ranked #1 By The ACSI®
"""

tv_ad_3 =\
"""
Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor
Shop Now
https://www.wayfair.com/furniture/free-shipping
AdFree Shipping on Orders Over $35. Shop Furniture, Home Décor, Cookware & More! Free Shipping on All Orders Over $35. Shop 55 Inch Tv, Home Décor, Cookware & More!
"""

Building a request for our agents to pick the best ad.

In [62]:
eval_request_msg = \
f"""
Can you evaluate these Bing ads for me? Which one convices you more to buy their particular offering? 
Select **ONLY** one. Please explain your reasoning, based on your financial situation, background and personality.

# AD 1
```
{tv_ad_1}
```

# AD 2
```
{tv_ad_2}
```

# AD 3
```
{tv_ad_3}
```
"""

print(eval_request_msg)


Can you evaluate these Bing ads for me? Which one convices you more to buy their particular offering? 
Select **ONLY** one. Please explain your reasoning, based on your financial situation, background and personality.

# AD 1
```

The Best TV Of Tomorrow - LG 4K Ultra HD TV
https://www.lg.com/tv/oled
AdThe Leading Name in Cinematic Picture. Upgrade Your TV to 4K OLED And See The Difference. It's Not Just OLED, It's LG OLED. Exclusive a9 Processor, Bringing Cinematic Picture Home.

Infinite Contrast · Self-Lighting OLED · Dolby Vision™ IQ · ThinQ AI w/ Magic Remote

Free Wall Mounting Deal
LG G2 97" OLED evo TV
Free TV Stand w/ Purchase
World's No.1 OLED TV

```

# AD 2
```

The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More
https://www.samsung.com
AdFrom 4K To 8K, QLED To OLED, Lifestyle TVs & More, Your Perfect TV Is In Our Lineup. Experience Unrivaled Technology & Design In Our Ultra-Premium 8K & 4K TVs.

Discover Samsung Event · Real Depth Enhancer · Anti-Reflection · 48 mo

Defining a Reason for them to require a new TV.

In [63]:
situation = "Your TV broke and you need a new one. You search for a new TV on Bing."

### Try with standard agents

To begin with, let's pick a pre-defined agent and ask him or her to perform the evaluations. To make it easier to change the chosen agent, we assign it to a variable first.

In [64]:
TinyPerson.all_agents

{'Sai Abhishek': TinyPerson(name='Sai Abhishek'),
 'Lisa': TinyPerson(name='Lisa')}

In [65]:
if "Lisa" in TinyPerson.all_agents:
    del TinyPerson.all_agents["Lisa"]

In [66]:
lisa = create_lisa_the_data_scientist()

In [67]:
lisa.change_context(situation)

In [68]:
TinyPerson.all_agents

{'Sai Abhishek': TinyPerson(name='Sai Abhishek'),
 'Lisa': TinyPerson(name='Lisa')}

In [69]:
lisa.listen_and_act(eval_request_msg)

USER --> Lisa: [CONVERSATION] 
          >  Can you evaluate these Bing ads for me? Which one convices you more to buy their
          > particular offering?  Select **ONLY** one. Please explain your reasoning, based on your
          > financial situation, background and personality.  # AD 1 ```  The Best TV Of Tomorrow -
          > LG 4K Ultra HD TV https://www.lg.com/tv/oled AdThe Leading Name in Cinematic Picture.
          > Upgrade Your TV to 4K OLED And See The Difference. It's Not Just OLED, It's LG OLED.
          > Exclusive a9 Processor, Bringing Cinematic Picture Home.  Infinite Contrast · Self-
          > Lighting OLED · Dolby Vision™ IQ · ThinQ AI w/ Magic Remote  Free Wall Mounting Deal LG
          > G2 97" OLED evo TV Free TV Stand w/ Purchase World's No.1 OLED TV  ```  # AD 2 ```  The
          > Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More https://www.samsung.com AdFrom 4K
          > To 8K, QLED To OLED, Lifestyle TVs & More, Your Perfect TV Is In Our Lineup. Experience
          > Unrivaled Technology & Design In Our Ultra-Premium 8K & 4K TVs.  Discover Samsung Event
          > · Real Depth Enhancer · Anti-Reflection · 48 mo 0% APR Financing  The 2 (...)

Lisa acts: [THINK] 
          > I need to evaluate these ads based on my financial situation, background, and
          > personality. I should consider what appeals to me personally and what fits my current
          > needs.

Lisa acts: [THINK] 
          > Considering my financial situation, I should be mindful of my budget. The Wayfair ad
          > offers free shipping on orders over $35, which is appealing for someone looking to save
          > money. However, it doesn't specify the quality or features of the TV, which is important
          > to me as a data scientist who values technology and innovation.

Lisa acts: [THINK] 
          > The LG ad emphasizes cinematic picture quality and advanced technology, which aligns
          > with my interest in high-quality visuals and technology. However, it might be more
          > expensive than I am willing to spend right now.

Lisa acts: [THINK] 
          > The Samsung ad offers a wide range of TVs with advanced features like Real Depth
          > Enhancer and Anti-Reflection, which are appealing to me as someone who appreciates
          > technology. Additionally, the 48-month 0% APR financing could make it more affordable.

Lisa acts: [TALK] 
          > After evaluating the ads, I am most convinced by the Samsung ad. It offers a range of
          > advanced features like Real Depth Enhancer and Anti-Reflection, which align with my
          > interest in technology. Additionally, the 48-month 0% APR financing makes it more
          > financially accessible for me.

Lisa acts: [DONE] 

Let's extract from the agent's interaction the best ad chosen. In this manner, we can easily process results later.

In [70]:
extractor = ResultsExtractor()

extraction_objective="Find the ad the agent chose. Extract the Ad number and title."

res = extractor.extract_results_from_agent(lisa, 
                          extraction_objective=extraction_objective,
                          situation=situation,
                          fields=["ad_number", "ad_title"],
                          verbose=True)

res

Extraction raw result message: {'content': '```json\n{"ad_number": "2", "ad_title": "The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More"}\n```', 'refusal': None, 'role': 'assistant'}


{'ad_number': '2',
 'ad_title': 'The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More'}

We can then easily get the ad number and title from the results:

In [71]:
f"{res['ad_number']}: {res['ad_title']}"

'2: The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More'

### Try with agents generated on the fly too

We don't really need to spend a lot of time customizing agents. We can create them on the fly from simple descriptions.

In [72]:
factory = TinyPersonFactory("""
                            People with a broad and diverse range of personalities, interests, backgrounds and socioeconomic status.
                            Focus in particular:
                              - on financial aspects, ensuring we have both people with high and low income.
                              - on aesthetic aspects, ensuring we have people with different tastes.
                            """)

In [21]:
people = factory.generate_people(10, verbose=True)
# verbose = True means
# Shows detailed output about each person being created
# Displays progress information
# Prints status messages during generation

Generated person 1/10: Carlos Mendoza is a 52 year old Street Vendor, Mexican, currently living in Mexico. Carlos Mendoza is a dedicated and resilient individual, known for his warm and friendly demeanor that draws customers to his taco stand. Despite the challenges of rising ingredient costs, he remains frugal and resourceful, always finding ways to manage his finances effectively. Carlos is passionate about cooking and often experiments with new recipes, hoping to expand his menu and attract more customers through social media marketing. In his personal life, he cherishes time spent with his family and enjoys playing soccer with friends on weekends, balancing his hardworking nature with moments of joy and relaxation.
Generated person 2/10: Aisha Thompson is a 29 year old Freelance Graphic Designer, American, currently living in United States. Aisha Thompson thrives in the dynamic world of freelance graphic design, where her creativity and adaptability shine. Her mornings begin with a

In [23]:
for person in people:
    person.listen_and_act(eval_request_msg)
    print("---------------------")

USER --> Carlos Mendoza: [CONVERSATION] 
          >  Can you evaluate these Bing ads for me? Which one convices you more to buy their
          > particular offering?  Select **ONLY** one. Please explain your reasoning, based on your
          > financial situation, background and personality.  # AD 1 ```  The Best TV Of Tomorrow -
          > LG 4K Ultra HD TV https://www.lg.com/tv/oled AdThe Leading Name in Cinematic Picture.
          > Upgrade Your TV to 4K OLED And See The Difference. It's Not Just OLED, It's LG OLED.
          > Exclusive a9 Processor, Bringing Cinematic Picture Home.  Infinite Contrast · Self-
          > Lighting OLED · Dolby Vision™ IQ · ThinQ AI w/ Magic Remote  Free Wall Mounting Deal LG
          > G2 97" OLED evo TV Free TV Stand w/ Purchase World's No.1 OLED TV  ```  # AD 2 ```  The
          > Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More https://www.samsung.com AdFrom 4K
          > To 8K, QLED To OLED, Lifestyle TVs & More, Your Perfect TV Is In Our Lineup. Experience
          > Unrivaled Technology & Design In Our Ultra-Premium 8K & 4K TVs.  Discover Samsung Event
          > · Real Depth Enhancer · Anti-Reflection · 48 mo 0% APR Financing  The 2 (...)

Carlos Mendoza acts: [THINK] 
                    > I need to evaluate these ads based on my financial situation, background, and
                    > personality. As a street vendor, my budget is limited, so affordability is a
                    > key factor. I also value practicality and reliability over luxury features.

Carlos Mendoza acts: [THINK] 
                    > Considering my financial situation, the Wayfair ad seems the most practical.
                    > It offers a 55-inch TV with free shipping on orders over $35, which is more
                    > affordable and suits my needs better than the high-end options from LG and
                    > Samsung.

Carlos Mendoza acts: [TALK] 
                    > Based on my financial situation and needs, I would choose the Wayfair ad. It
                    > offers a 55-inch TV with free shipping on orders over $35, which is more
                    > affordable and practical for me compared to the high-end options from LG and
                    > Samsung.

Carlos Mendoza acts: [DONE] 

...

Giovanni Russo acts: [THINK] 
                    > Ad 2 from Samsung offers a wide range of TVs with advanced technology and
                    > financing options. However, the focus on ultra-premium models might not align
                    > with my current financial priorities.

Giovanni Russo acts: [THINK] 
                    > Ad 3 from Wayfair focuses on affordability and free shipping, which is
                    > appealing. However, it doesn't emphasize the quality of the TV itself, which
                    > is important to me.

Giovanni Russo acts: [TALK] 
                    > After considering all the ads, I would choose Ad 1 from LG. It offers a high-
                    > quality OLED TV with features like Dolby Vision and a free wall mounting deal,
                    > which aligns with my appreciation for quality and value. As a restaurant
                    > owner, I value reliable products that enhance the experience, and this TV
                    > seems to fit that criteria.

Giovanni Russo acts: [DONE] 

---------------------


In [24]:
extractor = ResultsExtractor()
extraction_objective="Find the ad the agent chose. Extract the Ad number and title. Extract only ONE result."

choices =[]

for person in people:
    res = extractor.extract_results_from_agent(person,
                                    extraction_objective=extraction_objective,
                                    situation=situation,
                                    fields=["ad_number", "ad_title"],
                                    fields_hints={"ad_number": "Must be an integer, not a string."},
                                    verbose=True)

    choices.append(res)

Extraction raw result message: {'content': '{"ad_number": 3, "ad_title": "Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now"}', 'refusal': None, 'role': 'assistant'}
Extraction raw result message: {'content': '```json\n{"ad_number": 2, "ad_title": "The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More"}\n```', 'refusal': None, 'role': 'assistant'}
Extraction raw result message: {'content': '```json\n{"ad_number": 1, "ad_title": "The Best TV Of Tomorrow - LG 4K Ultra HD TV"}\n```', 'refusal': None, 'role': 'assistant'}
Extraction raw result message: {'content': '```json\n{"ad_number": 1, "ad_title": "The Best TV Of Tomorrow - LG 4K Ultra HD TV"}\n```', 'refusal': None, 'role': 'assistant'}
Extraction raw result message: {'content': '```json\n{"ad_number": 3, "ad_title": "Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now"}\n```', 'refusal': None, 'role': 'assistant'}
Extraction raw result message: {'content': '{"ad_number": 2, "ad_title": "The Full Samsung TV Lineup - Neo 

In [25]:
choices

[{'ad_number': 3,
  'ad_title': 'Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now'},
 {'ad_number': 2,
  'ad_title': 'The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More'},
 {'ad_number': 1, 'ad_title': 'The Best TV Of Tomorrow - LG 4K Ultra HD TV'},
 {'ad_number': 1, 'ad_title': 'The Best TV Of Tomorrow - LG 4K Ultra HD TV'},
 {'ad_number': 3,
  'ad_title': 'Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now'},
 {'ad_number': 2,
  'ad_title': 'The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More'},
 {'ad_number': 1, 'ad_title': 'The Best TV Of Tomorrow - LG 4K Ultra HD TV'},
 {'ad_number': 3,
  'ad_title': 'Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now'},
 {'ad_number': 3,
  'ad_title': 'Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now'},
 {'ad_number': 1, 'ad_title': 'The Best TV Of Tomorrow - LG 4K Ultra HD TV'}]

In [26]:
votes = {}
for choice in choices:
    print(f"{choice['ad_number']}: {choice['ad_title']}")

    ad_number = choice['ad_number']
    if ad_number not in votes:
        votes[ad_number] = 0
    votes[ad_number] += 1

3: Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now
2: The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More
1: The Best TV Of Tomorrow - LG 4K Ultra HD TV
1: The Best TV Of Tomorrow - LG 4K Ultra HD TV
3: Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now
2: The Full Samsung TV Lineup - Neo QLED, OLED, 4K, 8K & More
1: The Best TV Of Tomorrow - LG 4K Ultra HD TV
3: Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now
3: Wayfair 55 Inch Tv - Wayfair 55 Inch Tv Décor Shop Now
1: The Best TV Of Tomorrow - LG 4K Ultra HD TV


In [27]:
votes

{3: 4, 2: 2, 1: 4}

Finally, we pick the winner ad.

In [28]:
# picks the most voted ad
winner = max(votes, key=votes.get)
winner

3